In [1]:
import os
import tarfile
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
from scipy.misc import imread, imsave
import time

In [3]:
class DeepLabModel(object):
    """
    加载 DeepLab 模型；
    推断 Inference.
    """
    INPUT_TENSOR_NAME = 'ImageTensor:0'
    OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
    INPUT_SIZE = 513
    FROZEN_GRAPH_NAME = 'frozen_inference_graph'

    def __init__(self, tarball_path):
        """
        加载预训练模型
        """
        self.graph = tf.Graph()

        graph_def = None
        # Extract frozen graph from tar archive.
        tar_file = tarfile.open(tarball_path)
        for tar_info in tar_file.getmembers():
            if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
                file_handle = tar_file.extractfile(tar_info)
                graph_def = tf.GraphDef.FromString(file_handle.read())
                break

        tar_file.close()

        if graph_def is None:
            raise RuntimeError('Cannot find inference graph in tar archive.')

        with self.graph.as_default():
            tf.import_graph_def(graph_def, name='')

        self.sess = tf.Session(graph=self.graph)


    def run(self, image):
        """

        Args:
        image:  转换为PIL.Image 类,不能直接用图片，原始图片

        Returns:
        resized_image: RGB image resized from original input image.
        seg_map: Segmentation map of `resized_image`.
        """
        width, height = image.size
        resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
        target_size = (int(resize_ratio * width), int(resize_ratio * height))
        resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
        batch_seg_map = self.sess.run(self.OUTPUT_TENSOR_NAME,
                                      feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
        seg_map = batch_seg_map[0]
        return resized_image, seg_map

    
#===============================================================================================================
def create_pascal_label_colormap():
    colormap = np.zeros((256, 3), dtype=int)
    ind = np.arange(256, dtype=int)

    for shift in reversed(range(8)):
        for channel in range(3):
            colormap[:, channel] |= ((ind >> channel) & 1) << shift
            ind >>= 3
    return colormap

def label_to_color_image(label):
    if label.ndim != 2:
        raise ValueError('Expect 2-D input label')

    colormap = create_pascal_label_colormap()

    if np.max(label) >= len(colormap):
        raise ValueError('label value too large.')
    return colormap[label]

def load_model():
    model_path = './deeplabmodels/deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz'#'deeplab_model.tar.gz'
    MODEL = DeepLabModel(model_path)
    #print('model loaded successfully!')
    return MODEL




class render():
    def __init__(self, inputpic, modeldir = "./models/cubist.model", 
                 archdir = "./models/model.meta"):
        self.input = inputpic
        self.model = modeldir
        self.arch = archdir

        self.image = self.input.astype(np.float32)
        self.image = np.expand_dims(self.image, axis = 0)


    def run(self,session):

        self.saver = tf.train.import_meta_graph(self.arch, clear_devices = True)
        self.saver.restore(session, self.model)
        self.inputs = tf.get_collection("inputs")[0]
        self.outputs = tf.get_collection("output")[0]

        self.time_s = time.time()
        self.result = self.outputs.eval({self.inputs : self.image})
        self.result = np.clip(self.result, 0.0, 255.0).astype(np.uint8)
        self.result = np.squeeze(self.result, 0)
        self.time_t = time.time()
        print ("First time. Time used: ", self.time_t - self.time_s)

        self.time_s = time.time()
        self.result = self.outputs.eval({self.inputs : self.image})
        self.result = np.clip(self.result, 0.0, 255.0).astype(np.uint8)
        self.result = np.squeeze(self.result, 0)
        self.time_t = time.time()
        print ("Second time. Time used: ", self.time_t - self.time_s)

        return self.result
model = load_model()

In [4]:
time_1 = time.time()
src = Image.open('./images/person3.jpg')
resized_im, seg_map = model.run(src)
seg_image = label_to_color_image(seg_map).astype(np.uint8)

src = cv2.cvtColor(np.asarray(src),cv2.COLOR_RGB2BGR)
resized_im = cv2.cvtColor(np.asarray(resized_im),cv2.COLOR_RGB2BGR)
src_resized = cv2.resize(src,(resized_im.shape[1],resized_im.shape[0]))
bg_img = np.zeros_like(src_resized)



#============================================================================================
tf.reset_default_graph()
#title = "composition_vii.model"
title = "cubist.model"
#title = "feathers.model"
#title = "la_muse.model"
#title = "mosaic.model"
#title = "the_scream.model"
#title = "udnie.model"
#title = "wave.model"
class render():
    def __init__(self, inputpic, modeldir = "./models/"+title, 
                 archdir = "./models/model.meta"):
        self.input = inputpic
        self.model = modeldir
        self.arch = archdir

        self.image = self.input.astype(np.float32)
        self.image = np.expand_dims(self.image, axis = 0)


    def run(self,session):

        self.saver = tf.train.import_meta_graph(self.arch, clear_devices = True)
        self.saver.restore(session, self.model)
        self.inputs = tf.get_collection("inputs")[0]
        self.outputs = tf.get_collection("output")[0]

        self.time_s = time.time()
        self.result = self.outputs.eval({self.inputs : self.image})
        self.result = np.clip(self.result, 0.0, 255.0).astype(np.uint8)
        self.result = np.squeeze(self.result, 0)
        self.time_t = time.time()
        print ("First time. Time used: ", self.time_t - self.time_s)

        self.time_s = time.time()
        self.result = self.outputs.eval({self.inputs : self.image})
        self.result = np.clip(self.result, 0.0, 255.0).astype(np.uint8)
        self.result = np.squeeze(self.result, 0)
        self.time_t = time.time()
        print ("Second time. Time used: ", self.time_t - self.time_s)

        return self.result

render_test = render(src_resized)

session = tf.Session()
with session.as_default():
    styback = render_test.run(session)


    
    
#=====================================================================
result = np.zeros_like(src_resized)

styback = Image.fromarray(styback.astype('uint8')).convert('RGB')
styback = styback.resize((np.shape(result)[1],np.shape(result)[0]))
styback = np.array(styback)

result[seg_map > 0] = resized_im[seg_map > 0]
result[seg_map == 0] = styback[seg_map == 0]


time_2 = time.time()
print ("Second time. Time used: ", time_2 - time_1)


cv2.imshow(title,result)
cv2.waitKey()
cv2.destroyAllWindows()

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./models/cubist.model
First time. Time used:  0.8921918869018555
Second time. Time used:  0.04167318344116211
Second time. Time used:  3.9635934829711914
